In [ ]:
using Knet

# Preprocessing
not implemented

# Initialization
implemented

In [ ]:
# initialization of weights (w/o batchnorm)
function init_model()
    w =  map(atype, Any[ xavier(Float64,3,3,2,64),  zeros(Float64,1,1,64,1), # conv_1_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_1_2
            xavier(Float64,3,3,64,128), zeros(Float64,1,1,128,1), # conv_2_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1) , # conv_2_2
            xavier(Float64,3,3,128,256), zeros(Float64,1,1,256,1), # conv_3_1 
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_3_2
            xavier(Float64,3,3,256,512), zeros(Float64,1,1,512,1), # conv_4_1 
            xavier(Float64,3,3,512,512), zeros(Float64,1,1,512,1),  # conv_4_2
            xavier(Float64,3,3,768,256), zeros(Float64,1,1,256,1), #conv_5_1
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_5_2
            xavier(Float64,3,3,384,128), zeros(Float64,1,1,128,1), # conv_6_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1), # conv_6_2
            xavier(Float64,3,3,192,64), zeros(Float64,1,1,64,1), # conv_7_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_7_2
            xavier(Float64,1,1,64,2), zeros(Float64,1,1,2,1)]); # conv_7_3
    return w
end

In [ ]:
# initialization of weights (w/o batchnorm)
function init_model(depth, max_channels)
    # initialize w
    for layer in range(1,depth)
        
    w =  map(atype, Any[ xavier(Float64,3,3,2,64),  zeros(Float64,1,1,64,1), # conv_1_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_1_2
            xavier(Float64,3,3,64,128), zeros(Float64,1,1,128,1), # conv_2_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1) , # conv_2_2
            xavier(Float64,3,3,128,256), zeros(Float64,1,1,256,1), # conv_3_1 
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_3_2
            xavier(Float64,3,3,256,512), zeros(Float64,1,1,512,1), # conv_4_1 
            xavier(Float64,3,3,512,512), zeros(Float64,1,1,512,1),  # conv_4_2
            xavier(Float64,3,3,768,256), zeros(Float64,1,1,256,1), #conv_5_1
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_5_2
            xavier(Float64,3,3,384,128), zeros(Float64,1,1,128,1), # conv_6_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1), # conv_6_2
            xavier(Float64,3,3,192,64), zeros(Float64,1,1,64,1), # conv_7_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_7_2
            xavier(Float64,1,1,64,2), zeros(Float64,1,1,2,1)]); # conv_7_3
    return w
end

In [ ]:
# initialization of weights (w/ batchnorm)
function init_model_bn()
    # include m (moments for batchnorm)
    m = Any[bnmoments() for i = 1:n]# n = how often do we use BN?
    
    w =  map(atype, Any[ xavier(Float64,3,3,2,64),  zeros(Float64,1,1,64,1), # conv_1_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_1_2
            xavier(Float64,3,3,64,128), zeros(Float64,1,1,128,1), # conv_2_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1) , # conv_2_2
            xavier(Float64,3,3,128,256), zeros(Float64,1,1,256,1), # conv_3_1 
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_3_2
            xavier(Float64,3,3,256,512), zeros(Float64,1,1,512,1), # conv_4_1 
            xavier(Float64,3,3,512,512), zeros(Float64,1,1,512,1),  # conv_4_2
            xavier(Float64,3,3,768,256), zeros(Float64,1,1,256,1), #conv_5_1
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_5_2
            xavier(Float64,3,3,384,128), zeros(Float64,1,1,128,1), # conv_6_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1), # conv_6_2
            xavier(Float64,3,3,192,64), zeros(Float64,1,1,64,1), # conv_7_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_7_2
            xavier(Float64,1,1,64,2), zeros(Float64,1,1,2,1)]); # conv_7_3
    return w, m
    end

# model
IMPLEMENTED

In [ ]:
function predict(w,x_in)
    
    # TODO:
    # include batch norm: https://github.com/denizyuret/Knet.jl/blob/master/src/batchnorm.jl
    
    # contraction
    #block 1
    # 256x256x2 -> 256x256x64
    conv_1_1 = relu.(conv4(w[1],x_in, padding = 1) .+ w[2]) # bock 1 convolution 1
    # 256x256x64 -> 256x256x64 (conc with block 7)
    conv_1_2 = relu.(conv4(w[3],conv_1_1, padding = 1) .+ w[4]) # block 1 convolution 2, incl max pooling
    # 256-256-64 -> 128x128x64
    pool_1 = pool(conv_1_2)
    
    # block 2
    # 128x128x64 -> 128x128x128
    conv_2_1 = relu.(conv4(w[5],pool_1, padding = 1) .+ w[6])
    # 128x128x128 -> 128x128x128 (conc with block 6)
    conv_2_2 = relu.(conv4(w[7],conv_2_1, padding = 1) .+ w[8])
    # 128x128x128 -> 64x64x128
    pool_2 = pool(conv_2_2)
    
    #block 3
    # 64x64x128 -> 64x64x256
    conv_3_1 = relu.(conv4(w[9],pool_2, padding = 1) .+ w[10])
    #64x64x256 -> 64x64x256
    conv_3_2 = relu.(conv4(w[11],conv_3_1, padding = 1) .+ w[12])
    # 64x64x256 -> 32x32x256 (conc with block 5)
    pool_3 = pool(conv_3_2)
    
    # bottleneck 
    # block 4
    # 32x32x256 -> 32x32x512
    conv_4_1 = relu.(conv4(w[13], pool_3, padding = 1) .+ w[14])
    #32x32x512 -> 32x32x512
    conv_4_2 = relu.(conv4(w[15], conv_4_1, padding = 1) .+ w[16])
    
    # upsampling
    # block 5
    # upsampling/"deconvolution": 32x32x512 -> 64x64x512
    upsampling_kernel_5 = convert(KnetArray, bilinear(Float64,2,2,512,512))
    upsampling_5 = deconv4(upsampling_kernel_5, conv_4_2, padding = 1,stride = 2)
    # concatenation with conv_3_2: 64x64x512 -> 64x64x768
    conc_5 = convert(KnetArray,cat(convert(Array, upsampling_5), convert(Array, conv_3_2), dims = 3))
    # 64x64x768 -> 64x64x256
    conv_5_1 = relu.(conv4(w[17], conc_5, padding = 1) .+ w[18])
    # 64x64x256-> 64x64x256
    conv_5_2 = relu.(conv4(w[19], conv_5_1, padding = 1) .+ w[20])
    
    # block 6
    # 64x64x256 -> 128x128x256
    upsampling_kernel_6 = convert(KnetArray, bilinear(Float64,2,2,256,256))
    upsampling_6 = deconv4(upsampling_kernel_6, conv_5_2, padding = 1,stride = 2) 
    # 128x128x256 -> 128x128x384
    conc_6 = convert(KnetArray,cat(convert(Array,upsampling_6), convert(Array,conv_2_2), dims = 3)) # with conv_2_2
    #128x128x384 -> 128x128x128
    conv_6_1 = relu.(conv4(w[21], conc_6, padding = 1) .+ w[22])
    # 128x128x128 -> 128x128x128
    conv_6_2 = relu.(conv4(w[23], conv_6_1, padding = 1) .+ w[24])
    
    # block 7
    # 128x128x128 -> 256x256x128
    upsampling_kernel_7 = convert(KnetArray, bilinear(Float64,2,2,128,128))
    upsampling_7 = deconv4(upsampling_kernel_7, conv_6_2, padding = 1,stride = 2) 
    # 256x256x128 -> 256x256x192
    conc_7 = convert(KnetArray,cat(convert(Array, upsampling_7), convert(Array, conv_1_2), dims = 3))
    # 256x256x192 -> 256x256x64
    conv_7_1 = relu.(conv4(w[25], conc_7, padding = 1) .+ w[26])
    # 256x256x64 -> 256x256x64
    conv_7_2 = relu.(conv4(w[27], conv_7_1, padding = 1) .+ w[28])
    # 256x256x64 -> 256x256x2 (1x1 convolution)
    conv_7_3 = sigm.(conv4(w[29], conv_7_2).+ w[30])
    
    return conv_7_3
    

# loss
NOT IMPLEMENTED

In [ ]:
# dice loss
loss(w, x, ygold, predict) = nll(predict(w, x), ygold);
lossgradient = grad(loss);

# train
NOT IMPLEMENTED

In [ ]:
function train(w, dtrn, optim, predict; epochs=10)
    
    for epoch = 1:epochs
        for (x, y) in dtrn
            g = lossgradient(w, x, y, predict)
            update!(w, g, optim)
        end
    end
end

In [ ]:
optim(w; lr=0.1) = optimizers(w, Sgd;  lr=lr);

In [ ]:
function report(epoch, w, dtrn, dtst, predict)
    println((:epoch, epoch, :trn, accuracy(w, dtrn, predict), :tst, accuracy(w, dtst, predict)))
end

In [ ]:
w   = initweights(atype);
opt = optim(w);

if fast
    train(w, dtrn, opt, predict; epochs=nepochs)
else
    for epoch = 1:nepochs
        train(w, dtrn, opt, predict; epochs=1)
        report(epoch, w, dtrn, dtst, predict)
    end
end